In [6]:
import cx_Oracle

In [5]:
conn = cx_Oracle.connect('hr/hr@127.0.0.1:1521/XE')
cursor = conn.cursor()

In [13]:
cursor.execute('drop table crawling')

In [14]:
cursor.execute('''
CREATE table crawling(
    no    number(7)   not null primary key,
    name  varchar(32) not null,
    phone varchar(32) not null,
    email varchar(64) not null,
    rdate date        not null
)
''')

In [11]:
cursor.execute('drop sequence crawling_seq')

DatabaseError: ORA-02289: sequence does not exist

In [12]:
cursor.execute('''
    CREATE sequence crawling_seq
         start with 1
         increment by 1
         maxvalue 9999999
         cache 2
         nocycle
''')

In [1]:
#1
sql = '''
INSERT INTO crawling (no, name, phone, email, rdate)
VALUES (crawling_seq.nextval, :name, :phone, :email, sysdate)
'''

In [6]:
result = cursor.execute(sql, ('홍길동', '02-1111-1111', 'mail@mail.com'))

In [7]:
print(result)

None


In [9]:
cursor.execute(sql, ('나길순', '010-1111-1111', 'mail2@mail.com'))
cursor.execute(sql, ('다길동', '010-1111-1112', 'mail3@mail.com'))

In [10]:
conn.commit()

## 목록

In [11]:
sql = '''
SELECT no, name, phone, email, rdate
FROM crawling
ORDER BY no asc
'''

In [12]:
cursor.execute(sql)

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@127.0.0.1:1521/XE>>

In [13]:
rows = cursor.fetchall()

In [14]:
rows #리스트로 튜플 안에 넣어서 보여줌

[(1,
  '홍길동',
  '02-1111-1111',
  'mail@mail.com',
  datetime.datetime(2021, 4, 13, 9, 25, 25)),
 (2,
  '나길순',
  '010-1111-1111',
  'mail2@mail.com',
  datetime.datetime(2021, 4, 13, 9, 26, 45)),
 (3,
  '다길동',
  '010-1111-1112',
  'mail3@mail.com',
  datetime.datetime(2021, 4, 13, 9, 26, 45))]

In [15]:
for row in rows:
    fmt = "{}, {}, {}, {}, {}"
    print(fmt.format(row[0], row[1], row[2], row[3], row[4]))

1, 홍길동, 02-1111-1111, mail@mail.com, 2021-04-13 09:25:25
2, 나길순, 010-1111-1111, mail2@mail.com, 2021-04-13 09:26:45
3, 다길동, 010-1111-1112, mail3@mail.com, 2021-04-13 09:26:45


## 조회

In [17]:
sql = '''
SELECT no, name, phone, email, rdate
FROM crawling
WHERE no = :no
'''

In [18]:
cursor.execute(sql,(1,)) #(1,) : 튜플임을 나타내기 위함

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@127.0.0.1:1521/XE>>

In [19]:
row = cursor.fetchone()

In [30]:
fmt = "{}, {}, {}, {}, {}"
print(fmt.format(row[0], row[1], row[2], row[3], row[4]))

1, 홍길동, 02-1111-1111, mail@mail.com, 2021-04-13 09:25:25


## 조건 추가

In [23]:
sql = '''
SELECT no, name, phone, email, rdate
FROM crawling
WHERE email = :email and name = :name
'''

In [24]:
cursor.execute(sql, ('mail@mail.com', '홍길동'))

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@127.0.0.1:1521/XE>>

In [25]:
row = cursor.fetchone()
if row != None:
    fmt = "{}, {}, {}, {}, {}"
    print(fmt.format(row[0], row[1], row[2], row[3], row[4]))

1, 홍길동, 02-1111-1111, mail@mail.com, 2021-04-13 09:25:25


## 수정

In [27]:
sql = '''
UPDATE crawling
SET name = :name, phone = :phone, email = :email, rdate = sysdate
WHERE no = :no
'''

In [28]:
cursor.execute(sql, ('아로미', '000', '0000@mail.com', 1))

In [29]:
conn.commit()

## 삭제

In [31]:
sql = '''
DELETE FROM crawling
WHERE no = :no
'''

In [32]:
cursor.execute(sql, (1,))

In [33]:
conn.commit()

In [34]:
cursor.close()
conn.close() #이렇게 닫아주면 쥬피터에서 더이상 연동해서 사용할 수 없음
             #재연결 시켜줘야함

## pandas와 oracle 데이터 주고받기

In [1]:
!pip show pandas

Name: pandas
Version: 1.2.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: c:\programdata\anaconda3\envs\pydata\lib\site-packages
Requires: python-dateutil, pytz, numpy
Required-by: seaborn


In [2]:
!pip show cx_oracle

Name: cx-Oracle
Version: 8.1.0
Summary: Python interface to Oracle
Home-page: https://oracle.github.io/python-cx_Oracle
Author: "Anthony Tuininga",
Author-email: "anthony.tuininga@gmail.com",
License: BSD License
Location: c:\programdata\anaconda3\envs\pydata\lib\site-packages
Requires: 
Required-by: 


In [3]:
!pip show sqlalchemy

Name: SQLAlchemy
Version: 1.4.7
Summary: Database Abstraction Library
Home-page: http://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: c:\programdata\anaconda3\envs\pydata\lib\site-packages
Requires: greenlet, importlib-metadata
Required-by: 


In [12]:
# Oracle -> Pandas
import pandas as pd
from sqlalchemy import create_engine

In [16]:
conn = cx_Oracle.connect('hr/hr@127.0.0.1:1521/XE')

In [17]:
sql = '''
  SELECT no, name, phone, email, rdate
  FROM crawling
  ORDER BY no ASC
'''

In [18]:
df = pd.read_sql(sql, conn) #pd : 판다스 -> 판다스는 기본적으로 read_sql문을 갖고 있음
                            #데이터를 데이터 프레임 안에 데리고 오는 것
conn.close()

In [19]:
df #데이터 베이스에서 가져온 데이터 프레임

,NO,NAME,PHONE,EMAIL,RDATE
0,2,나길순,010-1111-1111,mail2@mail.com,2021-04-13 09:26:45
1,3,다길동,010-1111-1112,mail3@mail.com,2021-04-13 09:26:45


In [20]:
max_val = max(df['NO']) + 1 #데이터 프레임의 컬럼명은 대문자로 써야함
max_val

4

In [21]:
new_row = {"NO":max_val, "NAME":'아로미', "PHONE":'010-1111-1111', "EMAIL":'mail7@mail.com', "RDATE":'2021-04-13 10:35:00'}

In [22]:
df2 = df.append(new_row, ignore_index = True) #데이터 프레임 안에 넣음

In [23]:
df2 

,NO,NAME,PHONE,EMAIL,RDATE
0,2,나길순,010-1111-1111,mail2@mail.com,2021-04-13 09:26:45
1,3,다길동,010-1111-1112,mail3@mail.com,2021-04-13 09:26:45
2,4,아로미,010-1111-1111,mail7@mail.com,2021-04-13 10:35:00


In [26]:
#오라클 테이블 생성
conn = cx_Oracle.connect('hr/hr@127.0.0.1:1521/XE')
cursor = conn.cursor()

In [27]:
cursor.execute("drop table crawling")

In [28]:
cursor.execute('''
CREATE table crawling(
    no    number(7)   not null   primary key,
    name  varchar(32) not null,
    phone varchar(32) not null,
    email varchar(32) not null,
    rdate date        not null
)
''')

In [29]:
df2['RDATE'] = pd.to_datetime(df2['RDATE']) #문자열(object)을 날짜 형태 타입으로 변환

In [31]:
df2.info() #e데이터프레임 안의 데이터 타입을 보여주는 함수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   NO      3 non-null      int64         
 1   NAME    3 non-null      object        
 2   PHONE   3 non-null      object        
 3   EMAIL   3 non-null      object        
 4   RDATE   3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 248.0+ bytes


## df2를 crawling 테이블로 저장

In [40]:
# Pandas -> Oracle
engine = create_engine('oracle+cx_oracle://hr:hr@localhost:1521/?service_name=XE', echo=False)

In [42]:
df2.to_sql(name='crawling', con=engine, if_exists='append', index=False) #to_sql 함수를 이용해 크롤링이라는 테이블에 연결

In [43]:
#등록확인
sql = '''
SELECT no, name, phone, email, rdate
FROM crawling
ORDER BY no asc
'''

In [44]:
cursor.execute(sql)

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@127.0.0.1:1521/XE>>

In [45]:
rows = cursor.fetchall()

In [46]:
for row in rows:
    fmt = "{}, {}, {}, {}, {}"
    print(fmt.format(row[0], row[1], row[2], row[3], row[4]))

2, 나길순, 010-1111-1111, mail2@mail.com, 2021-04-13 09:26:45
3, 다길동, 010-1111-1112, mail3@mail.com, 2021-04-13 09:26:45
4, 아로미, 010-1111-1111, mail7@mail.com, 2021-04-13 10:35:00


In [47]:
#데이터프레임으로 오라클 데이터 가져오기
df3 = pd.read_sql(sql, conn)
df3

,NO,NAME,PHONE,EMAIL,RDATE
0,2,나길순,010-1111-1111,mail2@mail.com,2021-04-13 09:26:45
1,3,다길동,010-1111-1112,mail3@mail.com,2021-04-13 09:26:45
2,4,아로미,010-1111-1111,mail7@mail.com,2021-04-13 10:35:00
